## Metaclasses and Attributes
Metaclasses let you intercept Python's `class` statement and provide special behaviour each time a class is defined. However, dynamic attributes enable you to override objects and cause unexpected side effects.

### Item 44: Use Plain Attributes Instead of Setter and Getter Methods
In Python you don't need to implement explicit setter and getter methods. Instead you should always start your implementations with simple public attributes:

In [3]:
class Resistor:
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.currect = 0

r1 = Resistor(50e3)
r1.ohms = 10e3

These attributes make operations like incrementing in place natureal and clear:

In [4]:
r1.ohms += 5e3

Later, if you decide you need special behaviour when an attribute is set, you can migrate to the `@property` decorator and its corresponding `setter` attribute. Below a new subclass of `Resistor` is defined that allows the current to vary by assigning the `voltage` property. For this code to work, the names of both the setter and he getter methods must match the intended property name:

In [5]:
class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0

    @property
    def voltage(self):  # the getter
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms

Now assigning the voltage property will run the voltage setter which in turn will update the `current` attribute of the object to match:

In [6]:
r2 = VoltageResistance(1e3)
print(f"Before: {r2.current:.2f} amps")
r2.voltage = 10
print(f"After: {r2.current:.2f} amps")

Before: 0.00 amps
After: 0.01 amps


Another advantage of using a `setter` on a property is validation on values and type checking for values passed to the class or making the parent attributes immutable. Here, we define a class that ensures that all resistance values are above zero ohms:

In [7]:
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)

    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError(f"Ohms must be > 0; got {ohms}")
        self._ohms = ohms

An exeption is also raised if we pass an invalid value to the constructor:

In [8]:
BoundedResistance(-5)

ValueError: Ohms must be > 0; got -5

This is because `BoundedResistance.__init__` calls the constructor in `Resistor` which assigns `self.ohm` to -5 which in turn calls the `@ohms.setter` method.  
When using the `@property` methods to implement setters and getters, don't set other attributes in getter property methods. This will cause unexpected behaviour.

In [9]:
class MysteriousResistor(Resistor):
    @property
    def ohms(self):
        self.voltage = self._ohms * self.current
        return self._ohms
    
    
    @ohms.setter
    def ohms(self, ohms):
        self._ohms = ohms

Setting other attributes in getter property methods leads to extremely bizarre behaviour (see below). The best approach is to only modify related object states in `@property.setter` methods and avoid:
1. Importing modules dynamically
2. Running slow helper functions
3. Doing I/O
4. Making expensive database queries 

In [10]:
r7 = MysteriousResistor(10)
r7.current = 0.01
print(f"Before: {r7.voltage:.3f}")
r7.ohms
print(f"After: {r7.voltage: .2f}")

Before: 0.000
After:  0.10


### Item 45: Consider `@property` Instead of Refactoring Attributes
1. Use `@property` to give existing instance attributes new functionality.
2. Make incremental progress toward better data models by using `@property`
3. Consider refactoring a class and all call sites when you find yourself using `@property` too heavily.

### Item 46: Use Descriptors for Reusable `@property` Methods
A big problem with the `@property` built-in is reuse:
1. The methods it decorates cannot be reused for multiple attributes of the same class.
2. They also can't be reused by unrelated classes.
For example, the class below validates that the grade is a percentage:

In [12]:
class Homework:
    def __init__(self):
        self._grade = 0
    
    @property
    def grade(self):
        return self._grade
    
    @grade.setter
    def grade(self, value):
        if not 0 <= value <= 100:
            raise ValueError(
                "Grade must be between 0 and 100"
            )
        self._grade = value

g = Homework()
g.grade = 95

If we want to give students grades based on exams, we need to repeat a lot of boilerplate to check whether each exam's grade is a percentage. Also, if another class wants to reuse the percentage validation, it needs to be rewritten.  
The better way for reusability is `descriptor`. The `descriptor protocol` defines how attribute access is interpreted by the language. A descriptor class can provide `__get__` and `__set__` methods that let you reuses the grade validation behaviour without boilerplate.

In [14]:
class Grade:
    def __init__(self):
        self._value = 0
    
    def __get__(self, instance, instance_type):
        return self._value
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(
                "Grade must be between 0 and 100"
            )
        self._value = value
    
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

When a property is assigned:

In [15]:
exam = Exam()
exam.writing_grade = 40

it is interpreted as:

exam.__dict__["writing_grade"].__set(exam, 40)

When the property is retrieved:   
exam.writing_grade

exam.__dict__["writing_grade].__get__(exam, Exam)

However, there is a problem with this descriptor implementation is that the `Grade` instance is contructed once and shared across all `Exam` instances. To solve this, the `Grade` class should keep track of its value for each unique `Exam` instance. This can be done by saving the per-instance state in a dictionary. Also, to not have memory leak, i.e. make instances set their reference to zero when no longer in use, we can use `weakref.WeakKeyDictionary`.

In [1]:
from weakref import WeakKeyDictionary

class Grade:
    def __init__(self):
       self._values = WeakKeyDictionary()
    
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return self._values.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(
                "Grade must be between 0 and 100"
            )
        self._values[instance] = value
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()


first_exam = Exam()
first_exam.writing_grade = 82
second_exam = Exam()
second_exam.writing_grade = 75
print(f"First exam grade is {first_exam.writing_grade}")
print(f"Second exam grade is {second_exam.writing_grade}")

First exam grade is 82
Second exam grade is 75


### Item 47: Use `__getattr__, __getattribute__`, and `__setattr__` for Lazy Attributes
Python's object hooks make it easy to write generic code for gluing systems together. For example, say that we want to represent records in a database as Python objects. The database has a schema set already. The code that uses objexts corresponding to those records must also know what the database looks like. However, in Python, the code that connects Python objects to the database doesn't need to explicitly specify the schema of the records; it can be generic. How? Certainly not with plain instance attributes, `@property` methods, and descriptors can't do this because they all need to be defined in advance.   
This dynamic behaviour is possible with the `__getattr__` special method. If a class defines `__getattr__`, that method is called every time an attribute can't be found in an object's instance dictionary.

In [1]:
class LazyRecord:
    def __init__(self):
        self.exists = 5
    
    def __getattr__(self, name):
        value = f"Value for {name}"
        setattr(self, name, value)
        return value

data = LazyRecord()
print("Before: ", data.__dict__)
print("foo: ", data.foo)
print("After: ", data.__dict__)

Before:  {'exists': 5}
foo:  Value for foo
After:  {'exists': 5, 'foo': 'Value for foo'}


If we want transactions in this database system. The next time the user accesses a property, we want to know whether the corresponding record in the database is still valid and whether the transaction is still open. The `__getattr__` won't allow for this reliably because it will use the object's instance dictionary as the fast path for existing attributes. Another hook called `__getattributes__` can do this. This special method is called every time an attribute is accessed on an object, even in the cases where it does existin the attribute dictionary. This enables checking for global transaction state on every property access. It's important to note that such an operation can incur significant overhead and negatively impact performance, but sometimes it's worth it. 

In [2]:
class ValidatingRecord:
    def __init__(self):
        self.exists = 5
    
    def __getattribute__(self, name):
        print(f"* Called __getattributes__({name!r})")
        try:
            value = super().__getattribute__(name)
            print(f"* Found {name!r}, returning {value!r}")
            return value
        except AttributeError:
            value = f"Value for {name}"
            print(f"* Setting {name!r} to {value!r}")
            setattr(self, name, value)
            return value
        
data = ValidatingRecord()
print("exists: ", data.exists)
print("First foo: ", data.foo)
print("Second foo: ", data.foo)

* Called __getattributes__('exists')
* Found 'exists', returning 5
exists:  5
* Called __getattributes__('foo')
* Setting 'foo' to 'Value for foo'
First foo:  Value for foo
* Called __getattributes__('foo')
* Found 'foo', returning 'Value for foo'
Second foo:  Value for foo
